## Import
#### Importamos Pandas (Para generar el dataset) y otras herramientas.

In [29]:
import pandas as pd 
import csv 
import ast 
from datetime import datetime as dt
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

### Data Frame 'movies_dataset.csv'

In [30]:
dfmovies = pd.read_csv('C:/Users/Usuario/Documents/Programacion/Henry/TP1/movies_dataset.csv')
dfmovies

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18396\4144701810.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfmovies = pd.read_csv('C:/Users/Usuario/Documents/Programacion/Henry/TP1/movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [31]:
dfmovies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Se eliminan columnas innecesarias acorde a la consigna y se almacena la info en un nuevo Data Frame 'dfmovies_etl'

Se eliminan tambien filas con errores y duplicados

In [32]:
# Columnas a eliminar pedidas por el cliente
dfmovies_etl = dfmovies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

# Lista con ID de filas problematicas/inutilizables
todrop = [19730, 29503, 35587]
dfmovies_etl = dfmovies_etl.drop(todrop, axis=0) # Se eliminan las filas cullo ID se encuentre en la lista

# Se eliminan duplicados
dfmovies_etl.drop_duplicates(inplace=True)

# Siguiente codigo para hacer un print con el numero de datos eliminados en el proceso
numPreDrop = dfmovies.shape[0]
numAftDrop = dfmovies_etl.shape[0]
numDroped = numPreDrop - numAftDrop
print(f'Numero de registros antes de la limpieza:{numPreDrop} \nNumero de registros luego de la limpieza:{numAftDrop} \nSe eliminaron en total {numDroped} registros')

Numero de registros antes de la limpieza:45466 
Numero de registros luego de la limpieza:45450 
Se eliminaron en total 16 registros


Se corrige y optimiza el tipo de dato de las columnas no anidadas

In [33]:
dfmovies_etl['budget'] = dfmovies_etl['budget'].astype(int) # Int para reducir el peso y realizar operaciones
dfmovies_etl['id'] = dfmovies_etl['id'].astype(int) # Int para reducir el peso y realizar operaciones
dfmovies_etl['popularity'] = dfmovies_etl['popularity'].astype(float) # Float al ser un valor decimal, reducir el peso y que se puede utilizar para operaciones
dfmovies_etl['release_date'] = pd.to_datetime(dfmovies_etl['release_date'], errors='coerce') # Date time para utilizar en funciones
dfmovies_etl['revenue'] = dfmovies_etl['revenue'].fillna(0).astype(int) # Llenamos valores Na con 0 para convertir en Int para reducir el peso y realizar operaciones
dfmovies_etl['runtime'] = dfmovies_etl['runtime'].fillna(0).astype(int) # Llenamos valores Na con 0 para convertir en Int para reducir el peso y realizar operaciones
dfmovies_etl['vote_average'] = dfmovies_etl['vote_average'].astype(float) # Float al ser un valor decimal, reducir el peso y que se puede utilizar para operaciones
dfmovies_etl['vote_count'] = dfmovies_etl['vote_count'].fillna(0).astype(int) # Llenamos valores Na con 0 para convertir en Int para reducir el peso y realizar operaciones
dfmovies_etl['title'] = dfmovies_etl['title'].astype(str) # Str ya que es texto
dfmovies_etl['overview'] = dfmovies_etl['overview'].astype(str) # Str ya que es texto
dfmovies_etl['tagline'] = dfmovies_etl['tagline'].astype(str) # Str ya que es texto
dfmovies_etl['original_language'] = dfmovies_etl['original_language'].astype(str) # Str ya que es texto
dfmovies_etl['status'] = dfmovies_etl['status'].astype(str) # Str ya que es texto

#### Desanidado

Desanidado de 'belongs_to_collection'

In [34]:
# Declaramos una lista vacia que contendra los datos limpios
collection_name = []

# Iteramos todas las filas
for index, row in dfmovies_etl.iterrows():
    string = str(row['belongs_to_collection']).strip('{}') # Removemos las llaves
    lista = string.split(',') # Almacenamos cada elemento del string
    
# Condicional para manejar celdas vacias
    if len(lista) >= 2: 
        collection_name.append(str(lista[1]).replace(" 'name': ", "").strip('[""]').strip("''")) # Almacena y limpia el nombre de la coleccion

# Si la celda esta vacia se debe almacenar un elemento vacio para conservar la estructura de datos    
    else:
        collection_name.append('')

In [35]:
# Reemplazamos los datos de la columna por los de la lista

dfmovies_etl['belongs_to_collection'] = collection_name

Desanidado de 'genre'

In [36]:
# Declaramos lista vacia para nombres de género
all_genre_names = []

# Recorremos cada fila del DataFrame
for index, row in dfmovies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        genre_list = ast.literal_eval(str(row['genres']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        genre_list = []

    # Lista temporal para nombres de género en la fila actual
    genre_names = []

    # Extraemos los nombres de cada diccionario en la lista temporal
    for genre in genre_list:
        genre_names.append(genre.get('name', ''))

    # Añadimos las listas temporales a las listas finales
    all_genre_names.append(genre_names)

# Revisamos el resultado
all_genre_names

[['Animation', 'Comedy', 'Family'],
 ['Adventure', 'Fantasy', 'Family'],
 ['Romance', 'Comedy'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy'],
 ['Action', 'Crime', 'Drama', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Action', 'Adventure', 'Drama', 'Family'],
 ['Action', 'Adventure', 'Thriller'],
 ['Adventure', 'Action', 'Thriller'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Horror'],
 ['Family', 'Animation', 'Adventure'],
 ['History', 'Drama'],
 ['Action', 'Adventure'],
 ['Drama', 'Crime'],
 ['Drama', 'Romance'],
 ['Crime', 'Comedy'],
 ['Crime', 'Comedy', 'Adventure'],
 ['Action', 'Comedy', 'Crime'],
 ['Comedy', 'Thriller', 'Crime'],
 ['Drama', 'Thriller'],
 ['Action', 'Adventure', 'Crime', 'Thriller'],
 ['Drama', 'Fantasy', 'Science Fiction', 'Thriller'],
 ['Drama', 'Romance'],
 ['Drama'],
 ['Comedy', 'Drama', 'Family'],
 ['Drama', 'Romance'],
 ['Fantasy', 'Science Fiction', 'Adventure'],
 ['Drama', 'Crime'],
 ['Drama', 'Crime'],
 ['Science Fiction', 'Thriller', 'Mystery'],
 ['Romance

In [37]:
# Reemplazamos los datos de la columna por los del diccionario

dfmovies_etl['genres'] = all_genre_names

Desanidado de 'production_companies'

In [38]:
# Declaramos lista vacia para nombres de companias
all_companies_names = []

# Recorremos cada fila del DataFrame
for index, row in dfmovies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        companies_list = ast.literal_eval(str(row['production_companies']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        companies_list = []

    # Lista temporal para nombres de companias en la fila actual
    companies_names = []

    # Extraemos los nombres de cada diccionario en la lista temporal
    for companies in companies_list:
        companies_names.append(companies.get('name', ''))

    # Añadimos las listas temporales a las listas finales
    all_companies_names.append(companies_names)

# Revisamos el resultado
all_companies_names

Error en indice 19729 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002946DA75550> (Valor nulo).
Error en indice 29502 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002944F997410> (Valor nulo).
Error en indice 35586 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x00000294655712D0> (Valor nulo).


[['Pixar Animation Studios'],
 ['TriStar Pictures', 'Teitler Film', 'Interscope Communications'],
 ['Warner Bros.', 'Lancaster Gate'],
 ['Twentieth Century Fox Film Corporation'],
 ['Sandollar Productions', 'Touchstone Pictures'],
 ['Regency Enterprises', 'Forward Pass', 'Warner Bros.'],
 ['Paramount Pictures',
  'Scott Rudin Productions',
  'Mirage Enterprises',
  'Sandollar Productions',
  'Constellation Entertainment',
  'Worldwide',
  'Mont Blanc Entertainment GmbH'],
 ['Walt Disney Pictures'],
 ['Universal Pictures', 'Imperial Entertainment', 'Signature Entertainment'],
 ['United Artists', 'Eon Productions'],
 ['Columbia Pictures', 'Castle Rock Entertainment'],
 ['Columbia Pictures', 'Castle Rock Entertainment', 'Enigma Pictures'],
 ['Universal Pictures', 'Amblin Entertainment', 'Amblimation'],
 ['Hollywood Pictures', 'Cinergi Pictures Entertainment'],
 ['Le Studio Canal+',
  'Laurence Mark Productions',
  'Metro-Goldwyn-Mayer (MGM)',
  'Carolco Pictures'],
 ['Universal Pictures',

In [39]:
# Reemplazamos los datos de la columna por los del diccionario

dfmovies_etl['production_companies'] = all_companies_names

Desanidado de 'production_countries'

In [40]:
# Declaramos lista vacia para nombres de paises
all_countries_names = []

# Recorremos cada fila del DataFrame
for index, row in dfmovies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        countries_list = ast.literal_eval(str(row['production_countries']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        countries_list = []

    # Lista temporal para nombres de paises en la fila actual
    countries_names = []

    # Extraer los datos de cada diccionario en la lista
    for countries in countries_list:
        countries_names.append(countries.get('name', ''))  # Añadir el nombre

    # Añadir las listas temporales a las listas finales
    all_countries_names.append(countries_names)

# Revisamos el resultado
all_countries_names

Error en indice 19729 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002946E2D8ED0> (Valor nulo).
Error en indice 29502 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002946ABDCA10> (Valor nulo).
Error en indice 35586 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002946AE3F5D0> (Valor nulo).


[['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['Germany', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United Kingdom', 'United States of America'],
 ['United States of America'],
 ['France', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['France', 'Germany', 'Italy', 'United States of America'],
 ['France', 'United States of America'],
 ['United Kingdom', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['France', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['Italy'],
 ['United States of America'],
 ['United Kingdom', 'France'],
 ['France', 'Germany', 'Spain'],
 ['China', 

In [41]:
# Reemplazamos los datos de la columna por los del diccionario

dfmovies_etl['production_countries'] = all_countries_names

Desanidado de 'spoken_languages'

In [42]:
# Declaramos lista vacia para lenguajes
all_languages_names = []

# Recorremos cada fila del DataFrame
for index, row in dfmovies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        languages_list = ast.literal_eval(str(row['spoken_languages']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        languages_list = []

    # Lista temporal para lenguajes en la fila actual
    languages_names = []

    # Extraer los datos de cada diccionario en la lista
    for languages in languages_list:
        languages_names.append(languages.get('name', ''))  # Añadir el nombre

    # Añadir las listas temporales a las listas finales
    all_languages_names.append(languages_names)

# Revisamos el resultado
all_languages_names

Error en indice 19729 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002946BD01E50> (Valor nulo).
Error en indice 29502 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002946BE85390> (Valor nulo).
Error en indice 35586 al convertir la cadena en lista de diccionarios.
Error malformed node or string on line 1: <ast.Name object at 0x000002946BF80210> (Valor nulo).


[['English'],
 ['English', 'Français'],
 ['English'],
 ['English'],
 ['English'],
 ['English', 'Español'],
 ['Français', 'English'],
 ['English', 'Deutsch'],
 ['English'],
 ['English', 'Pусский', 'Español'],
 ['English'],
 ['English', 'Deutsch'],
 ['English'],
 ['English'],
 ['English', 'Latin'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', 'Deutsch'],
 ['English', 'Español', 'Nederlands'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['广州话 / 廣州話', 'Français'],
 ['普通话'],
 ['English'],
 ['English', 'Français'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['Pусский', 'English'],
 ['English'],
 ['Español', 'English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', 'Español'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['Español', 'English', 'Français', 'Magyar'],
 [],
 ['English'],
 ['shqip', 'Italiano'],
 ['English', 'Deutsch'],
 ['English'],
 ['Eng

In [43]:
# Reemplazamos los datos de la columna por los del diccionario

dfmovies_etl['spoken_languages'] = all_languages_names

### Nuevas columnas necesarias

Columna 'release_year'

In [44]:
# Extraemos los años desde la columna release_date a una lista
release_year = dfmovies_etl['release_date'].dt.year

# Reemplazamos los datos de la columna por los de la lista
dfmovies_etl['release_year'] = release_year

# Llenamos datos vacios con 0 y almacenamos formateamos los años como int para reducir peso
dfmovies_etl['release_year'] = dfmovies_etl['release_year'].fillna(0).astype(int)

Columna 'return'

In [45]:
# Declaramos una lista vacia
return_ = []

# Iteramos cada fila del Data Frame
for index, row in dfmovies_etl.iterrows():

    # Condicional para evitar diviciones por 0 y falta de datos
    if row['budget'] != 0: 
        return_.append(round((row['revenue'] / row['budget']), 2)) # Aplicamos funcion indicada por el cliente. Resultado floar redondeado a 2 decimales para mejor legibilidad

    # Se rellena los datos faltantes con 0 para conservar estructura de datos
    else:
        return_.append(0)

# Añadimos la lista creada como columna al Data Frame
dfmovies_etl['return']=return_

### Data Frame 'credits.csv'

Carga del CSV como Data Frame

In [46]:
dfcreditsetl = pd.read_csv('C:/Users/Usuario/Documents/Programacion/Henry/TP1/credits.csv')
dfcreditsetl

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


Desanidado de columna 'Cast'

In [47]:
# Creamos lista final para nombres del cast
all_cast_names = []

# Recorremos cada fila del Data Frame
for index, row in dfcreditsetl.iterrows():
    
    # Convertimos strings a lista de diccionarios
    try:
        cast_list = ast.literal_eval(str(row['cast']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        cast_list = []

    # Creamos una lista temporal para nombres de cast en la fila actual
    cast_names = []

    # Extraemos los datos de cada diccionario en la lista
    for cast in cast_list:
        cast_names.append(cast.get('name', ''))

    # Añadimos las listas temporales a las listas finales
    all_cast_names.append(cast_names)

# Chequeamos que el resultado sea correcto
all_cast_names

[['Tom Hanks',
  'Tim Allen',
  'Don Rickles',
  'Jim Varney',
  'Wallace Shawn',
  'John Ratzenberger',
  'Annie Potts',
  'John Morris',
  'Erik von Detten',
  'Laurie Metcalf',
  'R. Lee Ermey',
  'Sarah Freeman',
  'Penn Jillette'],
 ['Robin Williams',
  'Jonathan Hyde',
  'Kirsten Dunst',
  'Bradley Pierce',
  'Bonnie Hunt',
  'Bebe Neuwirth',
  'David Alan Grier',
  'Patricia Clarkson',
  'Adam Hann-Byrd',
  'Laura Bell Bundy',
  'James Handy',
  'Gillian Barber',
  'Brandon Obray',
  'Cyrus Thiedeke',
  'Gary Joseph Thorup',
  'Leonard Zola',
  'Lloyd Berry',
  'Malcolm Stewart',
  'Annabel Kershaw',
  'Darryl Henriques',
  'Robyn Driscoll',
  'Peter Bryant',
  'Sarah Gilson',
  'Florica Vlad',
  'June Lion',
  'Brenda Lockmuller'],
 ['Walter Matthau',
  'Jack Lemmon',
  'Ann-Margret',
  'Sophia Loren',
  'Daryl Hannah',
  'Burgess Meredith',
  'Kevin Pollak'],
 ['Whitney Houston',
  'Angela Bassett',
  'Loretta Devine',
  'Lela Rochon',
  'Gregory Hines',
  'Dennis Haysbert',
 

Desanidado de columna 'Crew', quedandonos solo con los directores

In [48]:
# Creamos lista final para nombres de directores
all_director_names = []

# Recorremos cada fila del DataFrame
for index, row in dfcreditsetl.iterrows():
    # Convertimos la celda en string y luego en lista de diccionarios (manejamos los errores para que continue el proceso)
    try:
        crew_list = ast.literal_eval(str(row['crew']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        crew_list = []

    # Creamos lista temporal para el nombre de los director en la fila actual
    director_names = []

    # Extraer el nombre de los directores de cada diccionario en la lista
    for crew_member in crew_list:
        if crew_member.get('job') == 'Director':
            director_name = crew_member.get('name', '')
            director_names.append(director_name)

    # Añadimos el nombre de los directores a la lista final
    all_director_names.append(director_names)

# Chequeamos que el resultado sea correcto
all_director_names

[['John Lasseter'],
 ['Joe Johnston'],
 ['Howard Deutch'],
 ['Forest Whitaker'],
 ['Charles Shyer'],
 ['Michael Mann'],
 ['Sydney Pollack'],
 ['Peter Hewitt'],
 ['Peter Hyams'],
 ['Martin Campbell'],
 ['Rob Reiner'],
 ['Mel Brooks'],
 ['Simon Wells'],
 ['Oliver Stone'],
 ['Renny Harlin'],
 ['Martin Scorsese'],
 ['Ang Lee'],
 ['Allison Anders',
  'Alexandre Rockwell',
  'Robert Rodriguez',
  'Quentin Tarantino'],
 ['Steve Oedekerk'],
 ['Joseph Ruben'],
 ['Barry Sonnenfeld'],
 ['Jon Amiel'],
 ['Richard Donner'],
 ['Victor Salva'],
 ['Mike Figgis'],
 ['Oliver Parker'],
 ['Lesli Linka Glatter'],
 ['Roger Michell'],
 ['Jean-Pierre Jeunet', 'Marc Caro'],
 ['Zhang Yimou'],
 ['John N. Smith'],
 ['Terry Gilliam'],
 ['Jean-Jacques Annaud'],
 ['Chris Noonan'],
 ['Christopher Hampton'],
 ['Tim Robbins'],
 ['Stephen Low'],
 ['Andy Tennant'],
 ['Amy Heckerling'],
 ['Darrell James Roodt'],
 ['Richard Loncraine'],
 ['Albert Hughes', 'Allen Hughes'],
 ['Michael Hoffman'],
 ['Paul W.S. Anderson'],
 ['Gu

Conformamos un nuevo Data Frame

In [49]:
# Le damos forma de Data Frame para poder hacer un merge con nuestro Data Frame final luego
dfcreditsetl['cast'] = all_cast_names
dfcreditsetl['crew'] = all_director_names
dfcreditsetl

,cast,crew,id
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch],15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker],31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer],11862
...,...,...,...
45471,"[Leila Hatami, Kourosh Tahami, Elham Korda]",[Hamid Nematollah],439050
45472,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Lav Diaz],111109
45473,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",[Mark L. Lester],67758
45474,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[Yakov Protazanov],227506


Left Join con el Data Frame 'dfmovies_etl'

In [50]:
# Asi se conservan la lista de actores y directores de peliculas que ya esten en el Data Frame
df_final_etl = pd.merge(dfmovies_etl,dfcreditsetl, on='id', how='left')
df_final_etl

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,[English],Released,nan,Toy Story,7.7,5415,1995,12.45,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]
1,,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413,1995,4.04,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston]
2,Grumpy Old Men Collection,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92,1995,0.00,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch]
3,,16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34,1995,5.09,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker]
4,Father of the Bride Collection,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173,1995,0.00,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45506,,0,"[Drama, Family]",439050,fa,Rising and falling between a man and woman.,0.072051,[],[Iran],NaT,...,[فارسی],Released,Rising and falling between a man and woman,Subdue,4.0,1,0,0.00,"[Leila Hatami, Kourosh Tahami, Elham Korda]",[Hamid Nematollah]
45507,,0,[Drama],111109,tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,...,[],Released,nan,Century of Birthing,9.0,3,2011,0.00,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Lav Diaz]
45508,,0,"[Action, Drama, Thriller]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,...,[English],Released,A deadly game of wits.,Betrayal,3.8,6,2003,0.00,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",[Mark L. Lester]
45509,,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,...,[],Released,nan,Satan Triumphant,0.0,0,1917,0.00,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[Yakov Protazanov]


Nos aseguramos que no se genere un valor faltante al crear la nueva columna

In [51]:
df_final_etl['cast'] = df_final_etl['cast'].apply(lambda x: x if isinstance(x, list) else [])
df_final_etl['crew'] = df_final_etl['crew'].apply(lambda x: x if isinstance(x, list) else [])

Realizamos una ultima limpieza

In [52]:
# Se encontraron valores repetidos en el Data Frame, identificamos las filas a eliminar en base al titulo

# Almacenamos la cantidad de filas antes de la limpieza
numPreDrop = df_final_etl.shape[0]

# Limpiamos valores duplicados
df_final_etl.drop_duplicates(subset=['id'], keep='first', inplace=True)

# Almacenamos la cantidad de filas luego de la limpieza y claculamos la diferencia
numAftDrop = df_final_etl.shape[0]
numDroped = numPreDrop - numAftDrop
print(f'Numero de registros antes de la limpieza:{numPreDrop} \nNumero de registros luego de la limpieza:{numAftDrop} \nSe eliminaron en total {numDroped} registros')

Numero de registros antes de la limpieza:45511 
Numero de registros luego de la limpieza:45433 
Se eliminaron en total 78 registros


In [53]:
df_final_etl

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,[English],Released,nan,Toy Story,7.7,5415,1995,12.45,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]
1,,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413,1995,4.04,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston]
2,Grumpy Old Men Collection,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92,1995,0.00,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch]
3,,16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34,1995,5.09,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker]
4,Father of the Bride Collection,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173,1995,0.00,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45506,,0,"[Drama, Family]",439050,fa,Rising and falling between a man and woman.,0.072051,[],[Iran],NaT,...,[فارسی],Released,Rising and falling between a man and woman,Subdue,4.0,1,0,0.00,"[Leila Hatami, Kourosh Tahami, Elham Korda]",[Hamid Nematollah]
45507,,0,[Drama],111109,tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,...,[],Released,nan,Century of Birthing,9.0,3,2011,0.00,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Lav Diaz]
45508,,0,"[Action, Drama, Thriller]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,...,[English],Released,A deadly game of wits.,Betrayal,3.8,6,2003,0.00,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",[Mark L. Lester]
45509,,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,...,[],Released,nan,Satan Triumphant,0.0,0,1917,0.00,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[Yakov Protazanov]


In [54]:
df_final_etl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45433 entries, 0 to 45510
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45433 non-null  object        
 1   budget                 45433 non-null  int32         
 2   genres                 45433 non-null  object        
 3   id                     45433 non-null  int32         
 4   original_language      45433 non-null  object        
 5   overview               45433 non-null  object        
 6   popularity             45430 non-null  float64       
 7   production_companies   45433 non-null  object        
 8   production_countries   45433 non-null  object        
 9   release_date           45346 non-null  datetime64[ns]
 10  revenue                45433 non-null  int32         
 11  runtime                45433 non-null  int32         
 12  spoken_languages       45433 non-null  object        
 13  status

# Generar archivo

In [55]:
parquet_filename = '../Datasets/dataset_peliculas.parquet'
table = pa.Table.from_pandas(df_final_etl)
pq.write_table(table, parquet_filename)